### import necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2 as cv
import random
import PIL
import pickle


### generate and preprocess the data

In [2]:
data = r"C:\Users\NARENDRA BABU\Desktop\dataset_train"
categories = ['Real', 'AiGenerated']

In [3]:
img_size = 48
training_data = []

i = 0
for category in categories:
    path = os.path.join(data,category)
    classes = categories.index(category)
    for img in os.listdir(path):
        i = i + 1
        img_array = cv.imread(os.path.join(path,img))
        new_array = cv.resize(img_array, (48,48))
        new_array = new_array/255
        training_data.append([new_array, classes])

In [4]:
len(training_data), training_data[0][0].shape

(100000, (48, 48, 3))

In [5]:
random.shuffle(training_data)

In [8]:
X_train = []
y_train = []

for features, label in training_data:
    X_train.append(features)
    y_train.append(label)

X_train = np.array(X_train).reshape(-1, img_size, img_size, 3)
y_train = np.array(y_train)

### Storing X_train and y_train

In [9]:
pickle_out = open("X_train.pickle", "wb")
pickle.dump(X_train, pickle_out, protocol=4)
pickle_out.close()

pickle_out = open("y_train.pickle", "wb")
pickle.dump(y_train, pickle_out, protocol=4)
pickle_out.close()

In [11]:
pickle_in = open("X_train.pickle", "rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y_train.pickle", "rb")
y_train = pickle.load(pickle_in)

### creating the model

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Sequential
model = keras.Sequential([
    keras.layers.Conv2D(32,(3,3), activation='relu', input_shape = (48,48,3)),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Dropout(0.2),
    
    keras.layers.Conv2D(64,(3,3), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Dropout(0.2),
    
    keras.layers.Conv2D(128,(3,3), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Dropout(0.2),
    
    keras.layers.Conv2D(256,(3,3), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Dropout(0.2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

C:\Users\NARENDRA BABU\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 2, 2, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 409,089 (1.56 MB)

 Trainable params: 409,089 (1.56 MB)

 Non-trainable params: 0 (0.00 B)

### training the model

In [15]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 189s 59ms/step - accuracy: 0.7496 - loss: 0.4959
Epoch 2/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 227s 73ms/step - accuracy: 0.8736 - loss: 0.2994
Epoch 3/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 195s 62ms/step - accuracy: 0.9002 - loss: 0.2461
Epoch 4/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 200s 64ms/step - accuracy: 0.9073 - loss: 0.2259
Epoch 5/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 231s 74ms/step - accuracy: 0.9151 - loss: 0.2111
Epoch 6/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 291s 83ms/step - accuracy: 0.9187 - loss: 0.1998
Epoch 7/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 205s 66ms/step - accuracy: 0.9244 - loss: 0.1919
Epoch 8/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 201s 64ms/step - accuracy: 0.9283 - loss: 0.1834
Epoch 9/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 218s 70ms/step - accuracy: 0.9332 - loss: 0.1701
Epoch 10/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 225s 72ms/step - accuracy: 0.9331 - loss: 0.1681
Epoch 11/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 219s 70ms/step - accuracy: 0.9361 - loss: 0.16

### saving model and loading 

In [25]:
model.save("AIGeneratedModel.h5")

In [26]:
model_new = keras.models.load_model("AIGeneratedModel.h5")

In [23]:
model_new.compiled_metrics == None

False

In [24]:
model_new.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

### test set and evaluation

In [28]:
data = r"C:\Users\NARENDRA BABU\Desktop\dataset_test"
categories = ['Real', 'AiGenerated']

img_size = 48
testing_data = []

i = 0
for category in categories:
    path = os.path.join(data,category)
    classes = categories.index(category)
    for img in os.listdir(path):
        i = i + 1
        img_array = cv.imread(os.path.join(path,img))
        new_array = cv.resize(img_array, (48,48))
        new_array = new_array/255
        testing_data.append([new_array, classes])
        
random.shuffle(testing_data)

X_test = []
y_test = []

for features, label in testing_data:
    X_test.append(features)
    y_test.append(label)

X_test = np.array(X_test).reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)

In [29]:
model_new.evaluate(X_test, y_test)

625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.9349 - loss: 0.1849


[0.18207883834838867, 0.9345499873161316]

In [30]:
y_pred = model_new.predict(X_test)

625/625 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step


In [31]:
y_predicted = []

for arr in y_pred:
    if arr[0] <= 0.5:
        y_predicted.append(0)
    else:
        y_predicted.append(1)

y_predicted = np.array(y_predicted)

In [32]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94     10000
           1       0.98      0.89      0.93     10000

    accuracy                           0.93     20000
   macro avg       0.94      0.93      0.93     20000
weighted avg       0.94      0.93      0.93     20000



### testing for a different path (img)

In [33]:
def find_out(path_img):
    img_arr = cv.imread(path_img)
    plt.imshow(img_arr)
    new_arr = cv.resize(img_arr, (48,48))
    new_arr = new_arr/255
    test = []
    test.append(new_arr)
    test = np.array(test).reshape(-1, img_size, img_size, 3)
    y = model_new.predict(test)
    if y[0] <= 0.5:
        print("The given image is Real.")
    else:
        print("The given image is AI Generated.")